### import

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm, trange
from math import exp, sqrt, log
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.special import i0, i1, iv
from numpy import random
from torch.nn.functional import normalize
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from torchsummary import summary

/home/reaganwu/miniconda3/envs/pntorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### The setup of the dataset

In [65]:
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)


### Define the model of the neural network

In [2]:
class BASICnet(nn.Module):
    def __init__(self, dim, width):
        super(BASICnet, self).__init__()
        self.dim = dim
        self.width = width

        self.fc1 = nn.Linear(self.dim, self.width)
        self.fc2 = nn.Linear(self.width, self.width)
        self.fc3 = nn.Linear(self.width, self.width)
        self.fc4 = nn.Linear(self.width, self.width)

        self.output = nn.Linear(self.width, 1, bias=True)

    def forward(self, x):
        # s = nn.ConstantPad2d(x, (0, self.dim - self.dim))
        y = self.fc1(x)
        y = torch.tanh(y)
        y = self.fc2(y)
        y = torch.tanh(y)
        y = self.fc3(y)
        y = torch.tanh(y)
        y = self.fc4(y)
        y = torch.tanh(y)
        
        output = self.output(y)
        return output

    def assign_value(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight.data)
                nn.init.constant_(m.bias.data, 0.1)

### Cauculate equation (Righthand side of the equation, RHS)

In [119]:
# def calculate_fx(x):
#     pi = torch.tensor(np.pi, dtype=torch.float32)
#     f_x = -160 * pi**2 * torch.prod(torch.sin(4 * pi * x), dim=-1)
#     return f_x

In [122]:
# import torch

# def custom_loss(inference, target, input_data):
#     # 计算模型输出与目标之间的均方误差
#     mse_loss = torch.nn.MSELoss()
#     loss = mse_loss(inference, target)

#     # 计算模型输出的二阶导数，需要再次使用 torch.autograd.grad
#     model_grads2 = torch.autograd.grad(inference, input_data, torch.ones_like(inference), create_graph=True)[0]

#     # 定义目标的二阶导数。在这里，您需要根据您的 PDE 和边界条件定义正确的二阶导数。
#     # 以下示例只是简单的示范，请根据问题进行调整。
#     target_grads2 = torch.ones_like(model_grads2)  # 这里将目标二阶导数设置为常数，您需要根据问题进行调整

#     # 计算二阶导数的均方误差
#     grad2_loss = mse_loss(model_grads2, target_grads2)

#     # 计算边界损失
#     boundary_loss = torch.mean(inference**2)

#     # 将模型输出误差、二阶导数误差和边界损失结合起来
#     total_loss = loss + grad2_loss + boundary_loss

#     return total_loss

### Generate the advice calculate_fx

In [3]:
def calculate_fx(X):
    # Check if there are boundary cases with 1 and -1
    has_boundary = torch.any((X == -1) | (X == 1))

    if has_boundary:
        # If there are boundary cases, set fx to zero
        fx = torch.tensor(0.0, dtype=torch.float32)
    else:
        # Calculate the regular fx
        fx = -160 * (torch.tensor(4 * np.pi, dtype=torch.float32).pow(2)) * torch.prod(torch.sin(4 * np.pi * X), dim=-1)

    return fx


### Generate model from the dataset and using montecarlo method to train the model

In [72]:
import random

def calculate_fx(X):
    # Check if there are boundary cases with 1 and -1 in each row
    has_boundary = torch.any((X == 1) | (X == -1), dim=-1)

    # Create an initial fx tensor filled with the regular values
    regular_fx = -160 * (torch.tensor(4 * np.pi, dtype=torch.float32).pow(2)) * torch.prod(torch.sin(4 * np.pi * X), dim=-1)
    
    # Where there are boundary cases, set fx to zero
    fx = torch.where(has_boundary, torch.tensor(0.0, dtype=torch.float32), regular_fx)

    return fx



def data_generator_monte_carlo(Numberofgenpoints, dim, prob_special = 0.1):
    source = torch.randn(size = (Numberofgenpoints, dim), requires_grad=True) # 生成一个大矩阵
    source = normalize(source, p=2)
    radius = torch.rand(size = (Numberofgenpoints, 1))
    radius = torch.pow(torch.rand(size = (Numberofgenpoints, 1)), 1/dim)
    source = source * radius

    # 随即确认替换的行
    num_replace_incol = torch.randint(1, Numberofgenpoints, size=(1,)).item()

    for _ in range(num_replace_incol):
        if random.random() < prob_special:
            special_value = random.choice([1, -1])
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            source[row_index] = torch.full((dim, ), special_value)
        else:
            row_index = torch.randint(0, Numberofgenpoints, size=(1,)).item()
            col_index = torch.randint(0, dim, size=(1,)).item()
            replace_value = torch.randint(0, 2, size=(1,)).item() * 2 - 1
            source[row_index, col_index] = replace_value

    max_value = torch.max(source)
    min_value = torch.min(source)

    target = calculate_fx(source)
    # set target as the requres_graade true

    if max_value > 1 or min_value < -1:
        raise ValueError
    
    return source, target

已经生成了包括边界条件的值。

In [25]:
# # class MyLoss(nn.Module):

# # 	def __init__(self):
# # 		# 超参数初始化，如
# # 		slef.param1 = 0
	
# # 	def forward(self, predict, label):  # 一般是预测值和label
# # 		# 进行损失计算，即前向传播，如
# # 		return torch.mean(torch.pow((predict - label), 2))  # 可以自己定义一些计算，但是所有的数学操作必须使用tensor提供的math。也可以用functional提供的一些损失计算，如交叉熵损失。

# class PDELoss(nn.Module):
#     def __init__(self):
#         super(PDELoss, self).__init__()

#     def forward(self, input, target):
#         # input.requires_grad = True
#         model_grads2 = torch.autograd.grad(input, target, torch.ones_like(input), create_graph=True, allow_unused=True)[0]

#         boundary_loss = torch.mean(input**2)
#         mse_loss = torch.nn.MSELoss()(input, target)

#         total_loss = model_grads2 + boundary_loss + mse_loss
#         return total_loss

In [59]:
# import torch
# import torch.nn as nn

# class CustomLoss(nn.Module):
#     def __init__(self):
#         super(CustomLoss, self).__init__()

#     def forward(self, u_pred, u_target):
#         # 偏微分方程损失
#         u_x = torch.autograd.grad(u_pred, u_pred, torch.ones_like(u_pred), create_graph=True)[0]
#         u_xx = torch.autograd.grad(u_x, u_pred, torch.ones_like(u_x), create_graph=True)[0]
#         u_xx_s = u_xx.sum(dim=-1, keepdim=True)
  
#         pde_loss = torch.mean((u_xx_s - u_target) ** 2)

#         # 边界损失
#         boundary_loss = torch.mean(u_pred**2)

#         # 平均绝对误差
#         mae_loss = torch.nn.L1Loss()
#         mae = mae_loss(u_xx_s, u_target)

#         total_loss = pde_loss + boundary_loss + 0.5 * mae
#         return total_loss


In [70]:
def pde_loss(input, data):
    # input.requires_grad = True
    model_grads2 = torch.autograd.grad(input, data, torch.ones_like(input), create_graph=True, allow_unused=True)[0]
    return model_grads2

def boundary_loss(input):
    return torch.mean(input**2)

def mse_loss(input, target):
    return torch.nn.MSELoss()(input, target)

### Dataset generate

In [73]:
test_gen = 100
test_dim = 10

test_source, test_target = data_generator_monte_carlo(test_gen, test_dim)

In [6]:
# GenPointsNum = 10000
Dim = 10
# source = data_generator_monte_carlo(GenPointsNum, Dim, 0.05)

## The optimizer and scheduler

In [60]:


LR = 1e-3
NUM_EPOCHS = 10000
BATCH_SIZE = 1000
loss_list = np.zeros(NUM_EPOCHS)
test_loss = np.zeros(NUM_EPOCHS)
max_value_loss = np.zeros(NUM_EPOCHS)
min_value_loss = np.zeros(NUM_EPOCHS)

net = BASICnet(dim=Dim, width=500)

# Optimizer and scheduler define there
Optimizer = optim.Adam(net.parameters(), lr=LR)
Scheduler = StepLR(Optimizer, step_size=1000, gamma=0.8)

# orignal print
print(sum(p.numel() for p in net.parameters() if p.requires_grad))
device = torch.device('cuda:0')
net.to(device)
summary(net, (Dim,))



757501
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 500]           5,500
            Linear-2                  [-1, 500]         250,500
            Linear-3                  [-1, 500]         250,500
            Linear-4                  [-1, 500]         250,500
            Linear-5                    [-1, 1]             501
Total params: 757,501
Trainable params: 757,501
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.89
Estimated Total Size (MB): 2.90
----------------------------------------------------------------


In [79]:
from torch.utils.data import DataLoader
from tqdm import trange

# data_loader = DataLoader(source, batch_size=BATCH_SIZE, shuffle=True)

def pde_loss(input, data, target_reshape):
    # 计算每行的梯度
    model_grads2 = torch.autograd.grad(input, data, torch.ones_like(input), create_graph=True, allow_unused=True)[0]

    # 计算每行的MSE，并将其累积
    pdeloss = sum([torch.nn.MSELoss()(grad, target) for grad, target in zip(model_grads2, target_reshape)])

    return pdeloss



for epoch in trange(NUM_EPOCHS):
    total_loss = 0.0
    gen_each_round = 1000
    grad_list = []
    data, target = data_generator_monte_carlo(gen_each_round, dim=Dim)
    data = data.to(device)
    target = target.to(device)

    out = net(data)
    out_reshape = torch.reshape(out, (-1,))
    target_reshape = torch.reshape(target, (-1,))
    
    pde_loss_ = pde_loss(out_reshape, data, target_reshape)
    boundary_loss_ = boundary_loss(out_reshape)
    mse_loss_ = mse_loss(out_reshape, target_reshape)
    loss = pde_loss_ + boundary_loss_ + mse_loss_

    Optimizer.zero_grad()
    loss.backward()
    Optimizer.step()
    Scheduler.step()

    if epoch % 10 == 0:  # Print every 10 epochs or adjust as needed
        print(f"Epoch {epoch}/{NUM_EPOCHS}, Loss: {loss.item()}")


  0%|          | 0/10000 [00:00<?, ?it/s]/home/reaganwu/miniconda3/envs/pntorch/lib/python3.8/site-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
  0%|          | 1/10000 [00:00<58:37,  2.84it/s]

Epoch 0/10000, Loss: 465284224.0


  0%|          | 12/10000 [00:02<30:53,  5.39it/s]

Epoch 10/10000, Loss: 316263008.0


  0%|          | 22/10000 [00:04<29:37,  5.61it/s]

...

 32%|███▏      | 3171/10000 [09:24<21:24,  5.32it/s]

Epoch 3170/10000, Loss: 576782080.0


 32%|███▏      | 3181/10000 [09:27<20:15,  5.61it/s]

Epoch 3180/10000, Loss: 481697472.0


KeyboardInterrupt: 